# RL SUMO Implementation
## By: Emanuel Figetakis 

In [1]:
import os 
import sys 
import optparse
import random
import time

> SUMO should be installed it:
>
> Ubuntu: ```sudo apt-get install sumo sumo-tools sumo-doc -y```
>
> Restart after installing


In [2]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("ENV ERROR")

In [3]:
from sumolib import checkBinary
import traci

In [4]:
def run(x):
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, '-c', 'highway.sumocfg',"--tripinfo-output", "tripinfo.xml", "-S", "--quit-on-end"]);
    step = 0
    x1 = x[0]
    y1 = x[1]
    dectstring = "det_" + str(x1) + "_" + str(y1)
    lanenum = "E" + str(x1)
    lanes=[0,1,2]
    lanes.remove(x1)
    laneoption1 = lanes[0]
    laneoption2 = lanes[1]
    while traci.simulation.getMinExpectedNumber() > 0:
        time.sleep(.005) ######### <------ Here you can adjust the Playback speed (helpful to lower for more iterations)
        traci.simulationStep()
        #print(step)
        det_vehs3 = traci.inductionloop.getLastStepVehicleIDs("det_4") ##ROADBLOCK
        det_vehs2 = traci.inductionloop.getLastStepVehicleIDs(dectstring) ###Decide to Move Lanes here 
        for V in det_vehs2: ### Run a Loop to change lanes 
            can_Merge = traci.vehicle.couldChangeLane(V,10)
            if can_Merge == True :
                try:
                    traci.vehicle.changeLane(V,laneoption1,25)
                    #print('C',V,laneoption1)
                except traci.exceptions.TraCIException:
                    traci.vehicle.changeLane(V,laneoption2,25)
                    #print('C',V,laneoption2)
            elif can_Merge == False:
                traci.vehicle.setStop(V,'E0',duration=1,startPos=280,pos=289,until=1)
                #print('Slowed')
                try:
                    traci.vehicle.changeLane(V,laneoption1,25)
                    #print('C',V,laneoption1)
                except traci.exceptions.TraCIException:
                    traci.vehicle.changeLane(V,laneoption2,25)
                    #print('C',V,laneoption2)
            #print(V,"Tried Merging ",traci.vehicle.couldChangeLane(V,10)) ###Results True/False if the Car can Change lanes
            #print("Change_Lane",traci.vehicle.changeLane(V,0,25)) ### Results if car can change lane 
        for vehs in det_vehs3:
            #print(vehs)
            try:
                traci.vehicle.setStop(vehs,'E0',duration=1,startPos=280,pos=289,until=1)
                traci.vehicle.changeLane(vehs, 2, 25)
            except traci.exceptions.TraCIException:
                print("VERY BAD")
        step += 1
        #print("Traci Expected Number", traci.simulation.getMinExpectedNumber())
        #print("Step",step)
    traci.close()
    sys.stdout.flush()

In [5]:
def runstandard():
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
    traci.close()
    sys.stdout.flush()

In [6]:
sumoBinary = checkBinary('sumo-gui')
run((1,5))  #######(You can decide here at which point you would like the agents to move lanes)

 Retrying in 1 seconds


In [7]:
from PTS_Int import docparse
x = docparse.getresults()
print("AVG Speed",x)

AVG Speed 2.106666666666667


In [8]:
import gym_env

env = gym_env.TrafficEnv1()

/home/manoli/Implicit-Sensing-Traffic/.venv/lib/python3.7/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [9]:
env.observation_space.sample()
states = env.observation_space.shape
action = env.action_space.n
print(states,action)

(1,) 12


In [10]:
episodes = 2
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
Episode:1 Score:1.2353300437220782
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
Episode:2 Score:2.0474082437063648


In [11]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
#from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import tensorflow.keras.callbacks 
from tqdm.keras import TqdmCallback

2025-02-06 13:02:26.809592: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:
2025-02-06 13:02:26.809634: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:
2025-02-06 13:02:26.809637: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/home/manoli/Implicit-Sensing-Traffic/.venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readt

In [12]:
def build_model(state,actions):
    model = Sequential()
    model.add(Dense(24,activation='relu', input_shape=states))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(action,activation='linear'))
    return model

In [13]:
states = env.observation_space.shape
action = env.action_space.n

model = build_model(states,action)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [14]:
def build_agent(model,action):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000,window_length=1)
    dqn = DQNAgent(model=model,memory=memory,policy=policy,nb_actions=action, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [16]:
dqn = build_agent(model,action)
dqn.compile(tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=['mae'])
#logger = tf.keras.callbacks.TensorBoard(log_dir='logs',write_graph=True,histogram_freq=5)
holder = dqn.fit(env,nb_steps=10,visualize=False,verbose=2)
#model.save('my_model.keras')

print(holder.history)
df1 = pd.DataFrame(holder.history)
df1.to_csv('test1.csv')

2025-02-06 13:03:19.724598: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2025-02-06 13:03:19.746598: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-02-06 13:03:19.752266: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3050 OEM computeCapability: 8.6
coreClock: 1.755GHz coreCount: 18 deviceMemorySize: 7.66GiB deviceMemoryBandwidth: 208.65GiB/s
2025-02-06 13:03:19.752337: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.8/lib64:
2025-02-06 13:03:19.752366: W tensorflow/stream_executor/platform/defa

Training for 10 steps ...
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 5/10: episode: 1, duration: 12.092s, episode steps:   5, steps per second:   0, episode reward:  1.273, mean reward:  0.255 [-0.127,  0.822], mean action: 6.200 [1.000, 11.000],  loss: --, mae: --, mean_q: --
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 Retrying in 1 seconds
 10/10: episode: 2, duration: 12.269s, episode steps:   5, steps per second:   0, episode reward:  2.082, mean reward:  0.416 [-0.155,  0.956], mean action: 5.200 [0.000, 11.000],  loss: --, mae: --, mean_q: --
done, took 24.362 seconds
{'episode_reward': [1.2726013931660392, 2.081537881527744], 'nb_episode_steps': [5, 5], 'nb_steps': [5, 10]}
